In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
!sm-docker --help

In [44]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not really used in this use case, use when need to store model artifacts (Ex: MME)
s3_bucket = 'spacy-sagemaker-us-east-1-bucket'

role = get_execution_role()

In [45]:
from time import gmtime, strftime

model_name = 'ons-mvp-entity-recognition-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://{}/spacy/'.format(s3_bucket) ## MODEL S3 URL
container = '{}.dkr.ecr.{}.amazonaws.com/ons-mvp-entity-recognition:latest'.format(account_id, region)
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: ons-mvp-entity-recognition-2022-08-24-18-00-32
Model data Url: s3://spacy-sagemaker-us-east-1-bucket/spacy/
Container image: 478704051461.dkr.ecr.us-east-1.amazonaws.com/ons-mvp-entity-recognition:latest
Model Arn: arn:aws:sagemaker:us-east-1:478704051461:model/ons-mvp-entity-recognition-2022-08-24-18-00-32


In [46]:
endpoint_config_name = 'ons-mvp-entity-recognition-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: ons-mvp-entity-recognition-config2022-08-24-18-00-34
Endpoint config Arn: arn:aws:sagemaker:us-east-1:478704051461:endpoint-config/ons-mvp-entity-recognition-config2022-08-24-18-00-34


In [50]:

%%time

import time

endpoint_name = 'ons-mvp-entity-recognition-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: ons-mvp-entity-recognition-endpoint2022-08-24-18-07-18
Endpoint Arn: arn:aws:sagemaker:us-east-1:478704051461:endpoint/ons-mvp-entity-recognition-endpoint2022-08-24-18-07-18
Endpoint Status: Creating
Waiting for ons-mvp-entity-recognition-endpoint2022-08-24-18-07-18 endpoint to be in service...
CPU times: user 63.6 ms, sys: 14.2 ms, total: 77.8 ms
Wall time: 2min


In [59]:
import json
content_type = "application/json"
request_body = {"input": "a uhe itaipu 60 hz deve ter a sua geração limitada"}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['output']
result

[['uhe itaipu', 'USINA'], ['60 hz', 'VALOR_COM_UNID.MEDIDA']]

In [63]:
response = sm_client.describe_endpoint(
    EndpointName=endpoint_name
)
response

{'EndpointName': 'ons-mvp-entity-recognition-endpoint2022-08-24-18-07-18',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:478704051461:endpoint/ons-mvp-entity-recognition-endpoint2022-08-24-18-07-18',
 'EndpointConfigName': 'ons-mvp-entity-recognition-config2022-08-24-18-00-34',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '478704051461.dkr.ecr.us-east-1.amazonaws.com/ons-mvp-entity-recognition:latest',
     'ResolvedImage': '478704051461.dkr.ecr.us-east-1.amazonaws.com/ons-mvp-entity-recognition@sha256:d7fabbe0739eb25d8f817c4166dda3b517ca19818fc5eece2ac585a38f7c88c2',
     'ResolutionTime': datetime.datetime(2022, 8, 24, 18, 7, 19, 422000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 8, 24, 18, 7, 18, 840000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 24, 